In [1]:
%reload_ext autoreload

# Imports

import os
import sys
module_path = os.path.abspath(os.path.join('../image_gen_dm')) # or the path to your source code
sys.path.append(str(module_path))

import tensorflow as tf
import tf_keras as tfk
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

tfkl = tfk.layers
tfpl = tfp.layers
tfd = tfp.distributions

TF_ENABLE_ONEDNN_OPTS=0
os.environ['TF_ENABLE_ONEDNN_OPTS'] = str(TF_ENABLE_ONEDNN_OPTS)
os.environ['TG_GPU_ALLOCATOR'] = 'cuda_malloc_async'

import denoising_vae as dvae
import denoising_vae.constants as consts

print('Tensorflow Version:', tf.__version__)
print("Available GPUs: ", len(tf.config.list_physical_devices('GPU')))

2024-05-05 10:27:50.828866: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 10:28:06.334627: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 10:28:06.511065: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 10:28:06.511174: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUM

Tensorflow Version: 2.16.1
Available GPUs:  1


In [2]:
# Constants

IMAGE_SIZE = 64
FLATTENED_SIZE = IMAGE_SIZE * IMAGE_SIZE * 3

ENCODER_HIDDEN_SIZES = [8192, 1792, 1152, 448]

DECODER_HIDDEN_SIZES = ENCODER_HIDDEN_SIZES[::-1]

LATENT_DIM = 2

MODEL_NAME = 'ncvae'
ENCODER_FILE_PATH = f'res/models/{MODEL_NAME}_encoder.h5'
DECODER_FILE_PATH = f'res/models/{MODEL_NAME}_decoder.h5'

LEARNING_RATE = 1e-4
VAL_PERCENTAGE = 0.1

In [ ]:
# Dataset loading

train_ds, val_ds = dvae.utils.load_datasets(val_percentage=VAL_PERCENTAGE)

In [3]:
# Model class

# This code is heavily based on the Keras Team's example on Convolutional VAEs with only minor modifications.
# The original code can be found at: https://github.com/keras-team/keras-io/blob/master/examples/generative/vae.py

# Author: [fchollet](https://twitter.com/fchollet)
# Licensed under the Apache License 2.0. See LICENSE file in the root directory for more information.


class Sampling(tfkl.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class NCVAE(tfk.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tfk.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tfk.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tfk.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            input_data, target_data = data
            noise_levels = tf.random.uniform(shape=(consts.BATCH_SIZE,), minval=0, maxval=consts.NOISE_LEVELS - 1, dtype=tf.int32)

            def apply_noise_to_image(image, noise_level):
                return dvae.utils.apply_noise(image, noise_level)
            
            input_data_noisy = tf.map_fn(
                lambda x: apply_noise_to_image(x[0], x[1]), 
                (input_data, noise_levels), dtype=tf.float32)

            z_mean, z_log_var, z = self.encoder(input_data_noisy)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.losses.binary_crossentropy(target_data, reconstruction), 
                    axis=(1,2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def call(self, input_data, training=False):
        _, _, z = self.encoder(input_data)
        reconstructed_image = self.decoder(z)

        return reconstructed_image


In [4]:
# Encoder

image_input = tfkl.Input(shape=consts.INPUT_SHAPE, name='image_input')

flattened_image = tfkl.Flatten()(image_input)

x = flattened_image
for layer_size in ENCODER_HIDDEN_SIZES:
    x = tfkl.Dense(layer_size, 
            activation='relu',
            kernel_initializer='glorot_uniform')(x)
    
z_mean = tfkl.Dense(LATENT_DIM, name='z_mean', kernel_initializer='glorot_uniform')(x)
z_log_var = tfkl.Dense(LATENT_DIM, name='z_log_var', kernel_initializer='glorot_uniform')(x)

z = Sampling()([z_mean, z_log_var])

encoder = tfk.Model(inputs=image_input, outputs=[z_mean, z_log_var, z], name='encoder')
encoder.summary()

# Decoder

latent_inputs = tfkl.Input(shape=(LATENT_DIM,))

y = latent_inputs
for layer_size in DECODER_HIDDEN_SIZES:
    y = tfkl.Dense(layer_size, 
            activation='relu',
            kernel_initializer='glorot_uniform')(y)
    
reconstructed_flattened_image = tfkl.Dense(FLATTENED_SIZE, kernel_initializer='glorot_uniform', activation='sigmoid')(y)
reshaped_image = tfkl.Reshape(consts.OUTPUT_SHAPE)(reconstructed_flattened_image)

decoder = tfk.Model(inputs=latent_inputs, outputs=reshaped_image, name='decoder')
decoder.summary()

# Model
        
model = NCVAE(encoder, decoder)
model.compile(optimizer=tfk.optimizers.Adam(learning_rate=LEARNING_RATE))


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 flatten (Flatten)           (None, 12288)                0         ['image_input[0][0]']         
                                                                                                  
 dense (Dense)               (None, 8192)                 1006714   ['flatten[0][0]']             
                                                          88                                      
                                                                                                  
 dense_1 (Dense)             (None, 1792)                 1468185   ['dense[0][0]']         

In [ ]:
# Load weights

if os.path.exists(ENCODER_FILE_PATH) and os.path.exists(DECODER_FILE_PATH):
    encoder.load_weights(ENCODER_FILE_PATH)
    decoder.load_weights(DECODER_FILE_PATH)

else:
    print('No weights found')

In [ ]:
# Run model

epochs = 4096

history = dvae.run_model(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, name=MODEL_NAME, plot=False, time=True)

file_path = f'res/histories/{MODEL_NAME}.csv'

history_df = pd.DataFrame(history.history)

if os.path.exists(file_path):
    df = pd.read_csv(file_path)

    last_time = df['elapsed_time'].iloc[-1]
    history_df['elapsed_time'] += last_time

    history_df.to_csv(file_path, mode='a', header=False, index=False)
else:
    history_df.to_csv(file_path, mode='w', header=True, index=False)

encoder.save_weights(ENCODER_FILE_PATH)
decoder.save_weights(DECODER_FILE_PATH)


In [ ]:
# Save model weights

encoder.save_weights(ENCODER_FILE_PATH)
decoder.save_weights(DECODER_FILE_PATH)

In [ ]:
# Load csv file

csv_file = f'res/histories/{MODEL_NAME}.csv'
df = pd.read_csv(csv_file)

In [ ]:
# Plot loss

plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.ylim(2555, 2590)

plt.plot(df['loss'], label='NCD-VAE', color='blue')
plt.legend(loc="upper left")

plt.show()

In [ ]:
# Plot elapsed time

plt.ylabel('Elapsed Time')
plt.xlabel('Epoch')

plt.plot(df['elapsed_time'], label='NCD-VAE', color='blue')

plt.legend(loc="upper left")

plt.show()

In [ ]:
# Validate model

dvae.val_model(model=model)

In [ ]:
# Plot latent space

dvae.plot_latent_space(model=model, start=-1, stop=1)